<td>
   <a target="_blank" href="https://www.clarifai.com/" ><img src="https://upload.wikimedia.org/wikipedia/commons/b/bc/Clarifai_Logo_FC_Web.png" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Clarifai/examples/blob/main/RAG/RAG.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Colab"></a>
</td>

# RAG

Retrieval Augmented Generation (RAG) is a method that improves the performance of Large Language Models (LLMs) by integrating an information retrieval component with the model's text generation capabilities. This approach addresses two main limitations of LLMs:

- Outdated Knowledge: Traditional LLMs, like ChatGPT, have a static knowledge base that ends at a certain point in time (for example, ChatGPT's knowledge cut-off is in January 2022). This means they lack information on recent events or developments.

- Knowledge Gaps and Hallucination: When LLMs encounter gaps in their training data, they may generate plausible but inaccurate information, a phenomenon known as "hallucination."

For more info on RAG: https://www.clarifai.com/blog/what-is-rag-retrieval-augmented-generation

This notebook contains
- How to setup RAG and upload, chat with the RAG interface.

# Table of Contents
* [Requirements](#requirements)
* [RAG Initialisation](#RAG_init)
* [RAG Setup](#RAG_setup)
* [Uploading Data](#upload_data)
    * [Data Overview](#data_overview)
* [Chat with your Data](#chat)
* [Optional: Explanation](#optional)
    * [RAG Prompter](#prompter)
    * [LLM](#llm_node)

## Requirements <a id="requirements"></a>

In [ ]:
!pip install -U clarifai

In [1]:
import os
#Replace your PAT
os.environ['CLARIFAI_PAT'] = "YOUR_PAT"

*Note: Guide to get your [PAT](https://docs.clarifai.com/clarifai-basics/authentication/personal-access-tokens)*

### For Colab

Note: To use the data from the example repo itself

In [ ]:
!git clone https://github.com/Clarifai/examples.git
%cd examples/datasets/upload/

## RAG Initialisation  <a id="RAG_init"></a>

In [ ]:
from clarifai.rag import RAG

# RAG setup  <a id="RAG_setup"></a>

Setting up a new RAG object. This will follow the following steps behind the scene
- Create a new App in Clarifai.
- Create a RAG prompter model.
- Create a model with the llm_url.
- Create a workflow with the RAG prompter and the LLM Model.

### LLM Models

User can choose different LLM Models for the RAG agent from [Clarifai Community Models](https://clarifai.com/explore/models?filterData=%5B%7B%22field%22%3A%22use_cases%22%2C%22value%22%3A%5B%22llm%22%5D%7D%5D&page=1&perPage=24)

Params for Retrieval of Data:
- min_score (float): The minimum score for search hits.
- max_results (float): The maximum number of search hits.
- prompt_template (str): The prompt template used. Must contain {data.hits} for the search hits and {data.text.raw} for the query string.


In [ ]:
USER_ID = 'user_id'
LLM_URL = 'https://clarifai.com/openai/chat-completion/models/GPT-4'
RAG_PROMPT_TEMPLATE = "Context information is below:\n{data.hits}\nGiven the context information and not prior knowledge, answer the query.\nQuery: {data.text.raw}\nAnswer: "
rag_object_gpt = RAG.setup(user_id=USER_ID,llm_url=LLM_URL, min_score=0.5, prompt_template=RAG_PROMPT_TEMPLATE)

### RAG initialisation from Workflow URL

If an RAG workflow is already created, we can directly use the Workflow URL

In [ ]:
WORKFLOW_URL = 'workflow_URL'
rag_object_from_url = RAG(workflow_url = WORKFLOW_URL)

# Upload Data <a id="upload_data"></a>

**The Clarifai Way**


Uploading Data into the App will do all this under the hood for the user,
- Data Upload to App in the Platform.
- Generate embeddings for the uploaded inputs.
- Store them in the Vector Database.
- Keep it ready for retrieval within seconds after upload

### Data Overview   <a id="data_overview"></a>

In [10]:
FILE_PATH = './data/paul_graham_essay.txt'
txt_file = open(FILE_PATH).read()

In [13]:
print(txt_file[:1000])



What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.

The language we used was an early version of Fortran. You had to type programs on punch cards, then stack them in t

### Uploading Textfile

In [ ]:
rag_object_gpt.upload(file_path=FILE_PATH,chunk_size= 1024) #parameters to split the document into chunks

### Supported formats for upload

- Files(Doc, PDF, Text)
- Entire folder of PDF, Doc, Text files
- URL of PDF,Doc, Text files

# Chat with your data  <a id="chat"></a>

In [16]:
rag_object_gpt.chat(messages=[{"role":"human", "content":"Who is Trevor Blackwell"}])

[{'role': 'assistant',
  'content': 'Trevor Blackwell is a person who was recommended by Robert to join a programming project. He was known for his plan to reduce everything in his life to a stack of notecards, which he carried around with him. Trevor proved to be a highly effective hacker. He wrote the manager part of the software which kept track of orders and statistics for an online store builder project.'}]

# Optional: Explanation  <a id="optional"></a>

## Rag Prompter      <a id="prompter"></a>

RAG Prompter is the retrieval part where the similar data to the query were retrieved from the Vector DB

In [18]:
from clarifai.client.model import Model

In [19]:
rag_prompter = Model(url='RAG_prompter_URL')

In [24]:
model_prediction = rag_prompter.predict_by_bytes(b'Who is Trevor Blackwell',input_type='text')
prompter_output = model_prediction.outputs[-1].data.text.raw
print(prompter_output)

Context information is below:
(Had I diligently set aside the proper proportion of the money I'd made consulting for Interleaf? No, I had not.) So although Robert had his graduate student stipend, I needed that seed funding to live on.

We originally hoped to launch in September, but we got more ambitious about the software as we worked on it. Eventually we managed to build a WYSIWYG site builder, in the sense that as you were creating pages, they looked exactly like the static ones that would be generated later, except that instead of leading to static pages, the links all referred to closures stored in a hash table on the server.

It helped to have studied art, because the main goal of an online store builder is to make users look legit, and the key to looking legit is high production values. If you get page layouts and fonts and colors right, you can make a guy running a store out of his bedroom look more legit than a big company.

(If you're curious why my site looks so old-fashion

## LLM Node   <a id="llm_node"></a>

The retrieved context is then sent to the LLM node the get the final response

In [23]:
LLM_URL = 'https://clarifai.com/openai/chat-completion/models/GPT-4'
LLM_Model = Model(url=LLM_URL)

In [30]:
llm_prediction = LLM_Model.predict_by_bytes(bytes(prompter_output, encoding='utf-8'),input_type='text')
print(llm_prediction.outputs[-1].data.text.raw)

Trevor Blackwell is a highly effective hacker who was recommended by Robert to join the programming team. He contributed significantly to the software development of the WYSIWYG site builder project.


## Clarifai Resources

**Website**: [https://www.clarifai.com](https://www.clarifai.com/)

**Demo**: [https://clarifai.com/demo](https://clarifai.com/demo)

**Sign up for a free Account**: [https://clarifai.com/signup](https://clarifai.com/signup)

**Developer Guide**: [https://docs.clarifai.com](https://docs.clarifai.com/)

**Clarifai Community**: [https://clarifai.com/explore](https://clarifai.com/explore)

**Python SDK Docs**: [https://docs.clarifai.com/python-sdk/api-reference](https://docs.clarifai.com/python-sdk/api-reference)

---